In [111]:
import re
import os
from os import path
import types
import nltk
import copy
import pandas as pd
from nltk.tokenize import sent_tokenize

def chapter_splitter(file, pattern):
    """The function splits up .txt utf-8 file into separate files by chapters and saves them 
    in corpus_data directory. To call the function, put a file-path and re-pattern as arguments. 
    Returns a generator with file-paths of the chapters for processing by other functions"""

    if path.exists('C:\\users\\admin\\corpus_data\\') == True: #To change the path and make it more flexibleS
        pass
    elif path.exists('C:\\users\\admin\\corpus_data\\') == False:
        os.mkdir('corpus_data')
    
    pattern = re.compile(pattern)

    with open(file, 'r', encoding='utf-8') as fp:
        doc = fp.readlines()
        n = 0 #Chapter number counter
        chapter_path = []
        for line in doc:
            if n == 0:
                output = open('corpus_data\intro.txt'.format(n), 'w', encoding='utf-8') #TO FIX INTRO!!
                output.write(line)
            matches = pattern.finditer(line)

            for match in matches:
                n += 1
                output = open('corpus_data\chapter_{}.txt'.format(n), 'w', encoding='utf-8')
                chapter_path += ['corpus_data\\chapter_{}.txt'.format(n)]
            output.write(line)

        output.close()

        for chapter in chapter_path: yield chapter #Generator with filepaths
    
ch = chapter_splitter('Fuji.txt', r'\d\.\d.?\s')



def sent_splitter(*args):
    """Sent_splitter can be used with chapter_splitter or without it.
    Splits text on sentences and saves them into a list. Takes on filepath as an argument (string)
    or multiple filepaths packed in a list of strings or a generator. Removes punctuation"""
    
    if isinstance(args, str): #TO DO!!!
        print('AAAA')
        with open(args, 'r', encoding='utf-8') as fp:
            data = fp.read()
            return sent_tokenize(data)

    
    elif isinstance(args, tuple):      
        print('The file type is a tuple')
        for t in args:
            for file in t:
                with open(file, 'r', encoding='utf-8') as fp:
                    data = fp.read()
                    yield sent_tokenize(data)

        
    elif isinstance(args, list):
        print('The file type is a list')
        for l in args:
            for file in l:
                with open(file, 'r', encoding='utf-8') as fp:
                    data = fp.read()
                    yield sent_tokenize(data)

    else:
        print('Unexpected error occured. The data type is a {}'.format(type(*args)))

sent = sent_splitter(ch)       
        
def clean_sentences(sentences):
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
                ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
                ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
                '[', '] ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '–', '…'))
    stopwords = nltk.corpus.stopwords.words('russian')
    
    memo_lemmas_copy = []
    for sentence in sentences:
        memo_lemmas_copy += [copy.copy(sentence)]
        
    memo_lemmas_copy = [(letter.replace('\n', ' ') for letter in sentence)
                        for sentence in memo_lemmas_copy]
    memo_lemmas_copy = [''.join(letter for letter in sentence if letter not in puncs)
                        for sentence in memo_lemmas_copy]
    df = pd.DataFrame(memo_lemmas_copy, columns=['text'])

    return df

clean_sentences(sent)


The file type is a tuple


,text
0,1.1.FUJIⓔ СТАРТАП Глотнув бондовского мартини...
1,"1.2.ТАНЯ Таня догадалась, что она красавица, ..."
2,1.3.FUJI-Ё СТАРТДАУН Федор Семенович в халате...
3,"2.1.ИНДИЙСКАЯ ТЕТРАДЬ.ДЖАНЫ Танек, смешная мо..."
4,"2.2 LAS NUEVAS CAZADORAS.ЖИЗЕЛЬ Таня – босая,..."
5,3.1.ИНДИЙСКАЯ ТЕТРАДЬ.ИНСАЙТЫ Танек!Опять гля...
6,3.2.LAS NUEVAS CAZADORAS.КЛАРИССА Жизель откр...
7,4.1 ИНДИЙСКАЯ ТЕТРАДЬ.БЕЛЫЙ ЗОНТ Присланная Д...
8,4.2.LAS NUEVAS CAZADORAS.ЗГЫЫН Таня с Кларисс...


<generator object sent_splitter at 0x000000001A760620>